# Leven at Newby Bridge using HBV model in eWaterCycle
This notebook demonstrates how eWaterCycle is used to run a model, in this case the classic HBV model, with forcing from the Caravan dataset. The only difference with the previous notebook is that here we use a "local" version of HBV, so HBV is not run inside of a container. This greatly speeds up the performance, but is less 'Reproducible' because now the results depend (more) on the setting / python version / dependcies of the local machine.

For this example, we chose a catchment in the Lake District in the UK. River discharge in river Leven is measured at the weirs at Newby Bridge, see the photo below. This observational data is available through the CamelsGB dataset [(Coxon, 2020)](https://essd.copernicus.org/articles/12/2459/2020/).
The larger [Caravan](https://www.nature.com/articles/s41597-023-01975-w) (collection of Camels...) dataset is available through eWaterCycle and will be used below.

![image](https://upload.wikimedia.org/wikipedia/commons/7/76/Weirs_on_the_River_Leven_at_Newby_Bridge_-_geograph.org.uk_-_5455774.jpg)

*Weirs on the River Leven at Newby Bridge by G Laird*

As a model we choose the classic [HBV model](https://hess.copernicus.org/articles/26/1371/2022/) for its simplicity. This nicely demonstrates how eWaterCycle works. More complex models are available through eWaterCycle, however:

- these often require parameter sets specific to a region
- these are more computationally intensive to run and therefore unsuited for a short 45 minute workshop.

Ask us about available models if you want to collaborate! 

### HBV model structure

The stucture of the HBV model is shown below.
Do note that the figure is missing a snow reservoir, however this has been implemented in the HBV model in this notebook.

![image](_images/model_layout.png)

*Image from the TU Delft course ENVM1502 - "River Basin Hydrology" by Markus Hrachowitz* 


## Starting up
To start up, we need to import eWaterCycle and a number of general Python libraries. 'Under the hood' eWaterCycle depends on a large number of other pieces of software, including but not limited to

- [grpc4bmi](https://github.com/eWaterCycle/grpc4bmi), a 'translator' for BMI function calls between different programming languages and across containers. This library was build by the eWaterCycle team, but is available openly for anyone that can benefit from its functionality. 
- apptainer, a container engine that runs the model-containers (Docker is supported too).
- [ESMValTool](https://github.com/ESMValGroup/ESMValTool), a climate data processing toolbox that originally intended to post-process climate data from CMIP projects for inclusion in IPCC reports, we adopted as tool for pre-processing climate data into forcing data for hydrological models
- Numerous hydrological models that are made available as plugins to eWaterCycle, see [eWaterCycle-leakybucket](https://github.com/eWaterCycle/ewatercycle-leakybucket) as an example. Note that plugins do not have to be owned and maintained by the eWaterCycle team: anyone with a model can make a plugin for eWaterCycle and make their model be available for others through the platform. 

Furthermore, eWaterCycle requires forcing data, obsrvational data and parameter sets to be available to users. If you want to install eWaterCycle on your own infrastructure, see the [eWaterCycle documentation](https://ewatercycle.readthedocs.io/en/latest/system_setup.html) or contact us.

In [40]:
# general python
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

#niceties
from rich import print

#needed
from ipywidgets import IntProgress
from IPython.display import display

In [41]:
# general eWaterCycle
import ewatercycle
import ewatercycle.models
import ewatercycle.forcing

## Choose region and time period: 
The HBV model is a lumped hydrological model. It considers a catchment as a homogenious area and calculates the major hydrological processes in it. It requires spatially aggregated rainfall and potential evaporation as inputs (ie. forcings). To calculate its modelled discharge at the outlet of the catchment it also needs a set of parameters. Usually these paramters are calibrated using (historical) observational data, so this is also required. 

in eWaterCycle we provide access to the Caravan dataset, which contains all of the above data for all the catchments in the different Camels datasets. In this notebook we use the precipitation and evaporation data from Caravan. However, there is a known problem with the caravan evaporation data and (the current version) shouldn't be used. In this notebook we do so to demonstrate how it works. It would be better to generate forcing from (for example) ERA5. [This](example_model_run_HBV_camels_catchment_ERA5_forcing.ipynb) additional notebook that looks very similar to this one explains how to do that. 

Using the interactive maps at [eWaterCycle caravan map](https://www.ewatercycle.org/caravan-map/) one can easily retrieve the identifier of the catchment.

Note that changing the region below will work, but that the parameters that are loaded later in this notebook are calibrated specifically for this particular catchment!

In [42]:
camelsgb_id = "lamah_208082"

We have to specify start and end date of the experiment that we want to do. For now we don't fuzz with diverences between calibration and validation periods (which officially of course is very bad...)

In [43]:
experiment_start_date = "1981-08-01T00:00:00Z"
experiment_end_date = "2010-08-31T00:00:00Z"

calibration_start_time = experiment_start_date
calibration_end_time = experiment_end_date

## Set up paths

Since forcing files are often re-used between experiments it is a best practice to save those intermediate files for re-use between experiments. These logical save-points in workflows are called 'rustpunten' in Dutch. It is important to store data in 'rustpunten' in standard formats. Working with clearly defined 'rustpunten' is a key element in the design of good workflows in general and was instrumental in designing eWaterCycle in particular. 

Here we set up some paths to store the forcing files we generate in your own home directory. 

To speed up this workshop, we have already created the forcing files in a central location, which we also create pointers to here. If you want to run for a different region, you will have to generate the forcing yourself. 

In [44]:
# Even though we don't use caravan data as forcing, we still call it forcing
# because it is generated using the forcing module of eWaterCycle
forcing_path_caravan = Path.home()/ "forcing" / camelsgb_id / "caravan"
forcing_path_caravan.mkdir(exist_ok=True, parents=True)

# If someone has prepared forcing for you, this path needs to be changed to that location. 
#prepared_forcing_path_caravan_central = Path("~/forcing/camelsgb_73010/caravan")

## Generate or load forcing
There are three options for creating forcing data objects:

- generate from climate data such as Caravan, ERA5 or CMIP. Note that if the directory you specify as destination already contains data this trying this will throw an error!
- load forcing data you generated previously by providing the location where it was stored
- load forcing data someone else (such as your teacher or a workshop leader) generated previously by providing the location where it was stored

First we will create a caravan forcing object, but as mentioned above, we will only use this for the discharge observations and the shape file of the region. After generating the object we show the fields it contains and plot the discharge data. 

The actual forcing object we will use in this example is the ERA5 based data.

For both caravan and ERA5 based forcing, only one of the options provided below should be used, use comments to select which one you want to use.

In [46]:
# option one: generate forcing data
camelsgb_forcing = ewatercycle.forcing.sources['CaravanForcing'].generate(
    start_time=experiment_start_date,
    end_time=experiment_end_date,
    directory=forcing_path_caravan,
    basin_id=camelsgb_id,
)

syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: <!DOCTYPE^ html PUBLIC "-//W3C//DTD XHTML 1.1//EN" "http://www.w3.org/TR/xhtml11/DTD/xhtml11.dtd"><html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en"> <head> <title>The page is temporarily unavailable</title> <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /> <style type="text/css"> /*<![CDATA[*/ body { background-color: #fff; color: #000; font-size: 0.9em; font-family: sans-serif,helvetica; margin: 0; padding: 0; } :link { color: #c00; } :visited { color: #c00; } a:hover { color: #f50; } h1 { text-align: center; margin: 0; padding: 0.6em 2em 0.4em; background-color: #900; color: #fff; font-weight: normal; font-size: 1.75em; border-bottom: 2px solid #000; } h1 strong { font-weight: bold; font-size: 1.5em; } h2 { text-align: center; background-color: #900; font-size: 1.1em; font-weight: bold; color: #fff; margin: 0; padding: 0.5em; border-bottom: 2px solid #000; } h3 { 

OSError: [Errno -70] NetCDF: DAP server error: 'https://opendap.4tu.nl/thredds/dodsC/data2/djht/ca13056c-c347-4a27-b320-930c2a4dd207/1/lamah.nc'

In [ ]:
# # option two or three: load data that you or someone else generated previously
# camelsgb_forcing = ewatercycle.forcing.sources['CaravanForcing'].load(directory=prepared_forcing_path_caravan_central)

In [7]:
print(camelsgb_forcing)

NameError: name 'camelsgb_forcing' is not defined

In [ ]:
#quick plot of the discharge data. 
ds_forcing = xr.open_mfdataset([camelsgb_forcing['Q'],camelsgb_forcing['pr'],camelsgb_forcing['evspsblpot']])
ds_forcing["Q"].plot()

In [ ]:
ds_forcing["pr"].plot(label = 'precipitation')
ds_forcing["evspsblpot"].plot(label = 'potential evaporation')
plt.legend()

## Load parameters from calibration

The HBV model contains five "stores" where the water is stored and nine parameters that control the flow between those stores and in and out of the model.
We have already calibrated the model for our region of choice and hardcoded those below. If you have changed the region and have calibrated for your region, you need to point to your calibration results here by uncommenting the load statement and pointing to the right file.

For the storages we can specify an array of starting values. If you don't the model will start 'empty' and needs some timesteps to 'fill up'. Especially for the rootzone storage it helps to not start empty. Note that all units are in mm: <br>

In [16]:
#we use the same initial conditions for all models in the ensemble

#               Si,  Su, Sf, Ss, Sp
s_0 = np.array([0,  100,  0,  5, 0])
S_names = ["Interception storage", "Unsaturated Rootzone Storage", "Fastflow storage", "Groundwater storage"]

#the names of the parameters are (luckily ;-) ) also constant for all models
p_names = ["$I_{max}$",  "$C_e$",  "$Su_{max}$", "β",  "$P_{max}$",  "$T_{lag}$",   "$K_f$",   "$K_s$", "FM"]
param_names = ["Imax","Ce",  "Sumax", "beta",  "Pmax",  "Tlag",   "Kf",   "Ks", "FM"]

In [17]:
#The number of ensemble members in our ensemble
N = 100

# p_min_initial= np.array([0,   0.2,  40,    .5,   .001,   1,     .01,  .0001,  .01])
# p_max_initial = np.array([8,    1,  800,   4,    .3,     10,    .1,   .01,  0.5])

p_min_initial= np.array([0,   0.2,  40,    0.5,   1,   1,     0.01,  0.0001,  0.01])
p_max_initial = np.array([50,    1.5,  3000,   30,    4,     50,    3,   3,  5])

parameters = np.zeros([9, N])

#here I use np.linspace to make a linear interpolation between the minimum and maximum parameters. Realize that this means that the first model will
#get all low parameters and the last model will get all high parameters. This can be done much smarter.
for param in range(9):
    parameters[param,:] = np.linspace(p_min_initial[param],p_max_initial[param],N)


In [18]:
# print(parameters.shape)  # Should be (expected_num_params, N)
print(parameters)
# type(parameters)


[[0.00000000e+00 5.05050505e-01 1.01010101e+00 1.51515152e+00
  2.02020202e+00 2.52525253e+00 3.03030303e+00 3.53535354e+00
  4.04040404e+00 4.54545455e+00 5.05050505e+00 5.55555556e+00
  6.06060606e+00 6.56565657e+00 7.07070707e+00 7.57575758e+00
  8.08080808e+00 8.58585859e+00 9.09090909e+00 9.59595960e+00
  1.01010101e+01 1.06060606e+01 1.11111111e+01 1.16161616e+01
  1.21212121e+01 1.26262626e+01 1.31313131e+01 1.36363636e+01
  1.41414141e+01 1.46464646e+01 1.51515152e+01 1.56565657e+01
  1.61616162e+01 1.66666667e+01 1.71717172e+01 1.76767677e+01
  1.81818182e+01 1.86868687e+01 1.91919192e+01 1.96969697e+01
  2.02020202e+01 2.07070707e+01 2.12121212e+01 2.17171717e+01
  2.22222222e+01 2.27272727e+01 2.32323232e+01 2.37373737e+01
  2.42424242e+01 2.47474747e+01 2.52525253e+01 2.57575758e+01
  2.62626263e+01 2.67676768e+01 2.72727273e+01 2.77777778e+01
  2.82828283e+01 2.87878788e+01 2.92929293e+01 2.97979798e+01
  3.03030303e+01 3.08080808e+01 3.13131313e+01 3.18181818e+01
  3.23232323e+01 3.28282828e+01 3.33333333e+01 3.38383838e+01
  3.43434343e+01 3.48484848e+01 3.53535354e+01 3.58585859e+01
  3.63636364e+01 3.68686869e+01 3.73737374e+01 3.78787879e+01
  3.83838384e+01 3.88888889e+01 3.93939394e+01 3.98989899e+01
  4.04040404e+01 4.09090909e+01 4.14141414e+01 4.19191919e+01
  4.24242424e+01 4.29292929e+01 4.34343434e+01 4.39393939e+01
  4.44444444e+01 4.49494949e+01 4.54545455e+01 4.59595960e+01
  4.64646465e+01 4.69696970e+01 4.74747475e+01 4.79797980e+01
  4.84848485e+01 4.89898990e+01 4.94949495e+01 5.00000000e+01]
 [2.00000000e-01 2.13131313e-01 2.26262626e-01 2.39393939e-01
  2.52525253e-01 2.65656566e-01 2.78787879e-01 2.91919192e-01
  3.05050505e-01 3.18181818e-01 3.31313131e-01 3.44444444e-01
  3.57575758e-01 3.70707071e-01 3.83838384e-01 3.96969697e-01
  4.10101010e-01 4.23232323e-01 4.36363636e-01 4.49494949e-01
  4.62626263e-01 4.75757576e-01 4.88888889e-01 5.02020202e-01
  5.15151515e-01 5.28282828e-01 5.41414141e-01 5.54545455e-01
  5.67676768e-01 5.80808081e-01 5.93939394e-01 6.07070707e-01
  6.20202020e-01 6.33333333e-01 6.46464646e-01 6.59595960e-01
  6.72727273e-01 6.85858586e-01 6.98989899e-01 7.12121212e-01
  7.25252525e-01 7.38383838e-01 7.51515152e-01 7.64646465e-01
  7.77777778e-01 7.90909091e-01 8.04040404e-01 8.17171717e-01
  8.30303030e-01 8.43434343e-01 8.56565657e-01 8.69696970e-01
  8.82828283e-01 8.95959596e-01 9.09090909e-01 9.22222222e-01
  9.35353535e-01 9.48484848e-01 9.61616162e-01 9.74747475e-01
  9.87878788e-01 1.00101010e+00 1.01414141e+00 1.02727273e+00
  1.04040404e+00 1.05353535e+00 1.06666667e+00 1.07979798e+00
  1.09292929e+00 1.10606061e+00 1.11919192e+00 1.13232323e+00
  1.14545455e+00 1.15858586e+00 1.17171717e+00 1.18484848e+00
  1.19797980e+00 1.21111111e+00 1.22424242e+00 1.23737374e+00
  1.25050505e+00 1.26363636e+00 1.27676768e+00 1.28989899e+00
  1.30303030e+00 1.31616162e+00 1.32929293e+00 1.34242424e+00
  1.35555556e+00 1.36868687e+00 1.38181818e+00 1.39494949e+00
  1.40808081e+00 1.42121212e+00 1.43434343e+00 1.44747475e+00
  1.46060606e+00 1.47373737e+00 1.48686869e+00 1.50000000e+00]
 [4.00000000e+01 6.98989899e+01 9.97979798e+01 1.29696970e+02
  1.59595960e+02 1.89494949e+02 2.19393939e+02 2.49292929e+02
  2.79191919e+02 3.09090909e+02 3.38989899e+02 3.68888889e+02
  3.98787879e+02 4.28686869e+02 4.58585859e+02 4.88484848e+02
  5.18383838e+02 5.48282828e+02 5.78181818e+02 6.08080808e+02
  6.37979798e+02 6.67878788e+02 6.97777778e+02 7.27676768e+02
  7.57575758e+02 7.87474747e+02 8.17373737e+02 8.47272727e+02
  8.77171717e+02 9.07070707e+02 9.36969697e+02 9.66868687e+02
  9.96767677e+02 1.02666667e+03 1.05656566e+03 1.08646465e+03
  1.11636364e+03 1.14626263e+03 1.17616162e+03 1.20606061e+03
  1.23595960e+03 1.26585859e+03 1.29575758e+03 1.32565657e+03
  1.35555556e+03 1.38545455e+03 1.41535354e+03 1.44525253e+03
  1.47515152e+03 1.50505051e+03 1.53494949e+03 1.56484848e+03
  1.59474747e+03 1.62464646e+03 1.65454545e+03 1.68444444e+03
  1.71434343e+03 1.74424242e+0

In [19]:
# # Zelf: Random Grid Search
# import itertools
# N = 100
# parameters = np.zeros([9, N])
# ensemble = []
# combinaties = itertools.product(*parameters)
# for counter in range(N):
#     parameters[ = np.random.uniform(p_min_initial, p_max_initial)

#    # print(counter, ":", random_params)
#     if counter % 10 == 0:
#        print(counter)
    
#     ensemble.append(ewatercycle.models.HBVLocal(forcing=camelsgb_forcing))
#     config_file, _ = ensemble[counter].setup(
#                             parameters = random_params,
#                             initial_storage=s_0,
#                             cfg_dir = "configFiles/hbv_ensembleMember_" + str(counter),
#                                )
#     ensemble[counter].initialize(config_file)

In [20]:
# # Zelf
# import itertools

# ensemble = []
# combinaties = itertools.product(*parameters)
# for counter, comb in enumerate(combinaties):
#     if counter % 10 == 0:
#         print(counter)
#     ensemble.append(ewatercycle.models.HBVLocal(forcing=camelsgb_forcing))
#     config_file, _ = ensemble[counter].setup(
#                             parameters = list(comb),
#                             initial_storage=s_0,
#                             cfg_dir = "configFiles/hbv_ensembleMember_" + str(counter),
#                                )
#     ensemble[counter].initialize(config_file)

In [21]:
N = 100

parameters = np.random.uniform(p_min_initial[:, np.newaxis], p_max_initial[:, np.newaxis], (len(p_min_initial), N))

In [22]:
ensemble = []

# for counter in range(N): 
#     ensemble.append(ewatercycle.models.HBVLocal(forcing=camelsgb_forcing))
#     config_file, _ = ensemble[counter].setup(
#                             parameters = ','.join([str(p) for p in parameters[:,counter]]),
#                             initial_storage=','.join([str(s) for s in s_0]),
#                             cfg_dir = "configFiles/hbv_ensembleMember_" + str(counter),
#                                )
#     ensemble[counter].initialize(config_file)

for counter in range(N): 
    ensemble.append(ewatercycle.models.HBVLocal(forcing=camelsgb_forcing))
    config_file, _ = ensemble[counter].setup(
                            parameters = parameters[:,counter],
                            initial_storage=s_0,
                            cfg_dir = "configFiles/hbv_ensembleMember_" + str(counter),
                               )
    ensemble[counter].initialize(config_file)


# for counter in range(N): 
#     ensemble.append(ewatercycle.models.HBVLocal(forcing=camelsgb_forcing))
#     config_file, _ = ensemble[counter].setup(
#     parameters=list(parameters[:, counter]),  # Pass as a list
#     initial_storage=list(s_0),  # Pass as a list
#     cfg_dir=f"configFiles/hbv_ensembleMember_{counter}",
# )
#     ensemble[counter].initialize(config_file)

NameError: name 'camelsgb_forcing' is not defined

## Observations and objective function

We will compare each model output to observations and we need some sort of objective function to judge if the output is any good and thus determine which parameters are good for this region. I provide a basic (bad!) objective function here.

Here we define the objective function

In [ ]:
def calibrationObjective(modelOutput, observation, start_calibration, end_calibration):
    # a function that takes in two dataFrames, interpolates the model output to the
    # observations and calculates the average absolute difference between the two.

    #combine the two in one dataFrame
    hydro_data = pd.concat([modelOutput.reindex(observation.index, method = 'ffill'), observation], axis=1)

    #only select the calibration period
    hydro_data = hydro_data[hydro_data.index > pd.to_datetime(pd.Timestamp(start_calibration).date())]
    hydro_data = hydro_data[hydro_data.index < pd.to_datetime(pd.Timestamp(end_calibration).date())]

    #calculate mean absolute difference

    #diff = hydro_data['Q'] - hydro_data['model output']

    #in kwadaat zodat pieken zwaarder wegen
    diff = (hydro_data['Q'] - hydro_data['model output']) ** 2
    absDiff = np.abs(diff)
    meanAbsDiff = np.mean(absDiff)

    return meanAbsDiff
  

Now we run the entire ensemble. Note that in theory this loop can be run in parallel. If you have access to many core (or a supercomputer), this loop can be speed up considerably! For HBV this is not really a problem, but when doing calibration with larger models, this is a must.

In [ ]:
#an object to show a progress bar, since this can take a while:
f = IntProgress(min=0, max=N) # instantiate the bar
display(f) # display the bar

#converting the observations to a pandas object, because Rolf is stupid
Q_pandas = ds_forcing["Q"].to_dataframe()

#an empty array to store the results in
objectives = []

#loop over all ensemble members
for ensembleMember in ensemble:
    Q_m = []
    time = []
    while ensembleMember.time < ensembleMember.end_time:
        ensembleMember.update()
        discharge_this_timestep = ensembleMember.get_value("Q")
        Q_m.append(discharge_this_timestep[0])
        time.append(pd.Timestamp(ensembleMember.time_as_datetime.date()))

    #calculate the objective function 
    discharge_dataframe = pd.DataFrame({'model output': Q_m}, index=pd.to_datetime(time))
    objective_this_model = calibrationObjective(discharge_dataframe,Q_pandas["Q"],calibration_start_time,calibration_end_time)
    objectives.append(objective_this_model)

    #it is good practice to remove any variable you don't need anymore to save memory.
    del Q_m, time, discharge_dataframe, objective_this_model
   
    #update progress bar
    f.value += 1

print(objectives)  

In [ ]:
#beste 10% zoeken

n = int(len(objectives) * 0.1)

lowest_indices = np.argsort(objectives)[:n]  # Sorteert en pakt de eerste n indices
type(lowest_indices)

# Stap 3: Haal de bijbehorende arrays op uit data_matrix
lowest_arrays = parameters[:, lowest_indices]

objectives = np.array(objectives)
# Print resultaten
print("Laagste 10% waarden in objectives:\n", objectives[lowest_indices])
print("\nBijbehorende arrays uit data_matrix:\n", lowest_arrays)

# # Sorteer de lijst en pak de eerste n elementen
# lowest_10_percent = sorted(objectives)[:n]

# print(lowest_10_percent)

# best_parameters = parameters[:, lowest_10_percent]

In [ ]:
# Bereken de minimum en maximum waarden per rij
p_min_initial = np.min(lowest_arrays, axis=0)  # Kleinste per rij
p_max_initial = np.max(lowest_arrays, axis=0)  # Grootste per rij

# Print resultaten
print("Minimumwaarden per rij:\n", p_min_initial)
print("\nMaximumwaarden per rij:\n", p_max_initial)

In [ ]:
N = 100

parameters = np.random.uniform(p_min_initial[:, np.newaxis], p_max_initial[:, np.newaxis], (len(p_min_initial), N))

In [ ]:
ensemble = []

# for counter in range(N): 
#     ensemble.append(ewatercycle.models.HBVLocal(forcing=camelsgb_forcing))
#     config_file, _ = ensemble[counter].setup(
#                             parameters = ','.join([str(p) for p in parameters[:,counter]]),
#                             initial_storage=','.join([str(s) for s in s_0]),
#                             cfg_dir = "configFiles/hbv_ensembleMember_" + str(counter),
#                                )
#     ensemble[counter].initialize(config_file)

for counter in range(N): 
    ensemble.append(ewatercycle.models.HBVLocal(forcing=camelsgb_forcing))
    config_file, _ = ensemble[counter].setup(
                            parameters = parameters[:,counter],
                            initial_storage=s_0,
                            cfg_dir = "configFiles/hbv_ensembleMember_" + str(counter),
                               )
    ensemble[counter].initialize(config_file)


# for counter in range(N): 
#     ensemble.append(ewatercycle.models.HBVLocal(forcing=camelsgb_forcing))
#     config_file, _ = ensemble[counter].setup(
#     parameters=list(parameters[:, counter]),  # Pass as a list
#     initial_storage=list(s_0),  # Pass as a list
#     cfg_dir=f"configFiles/hbv_ensembleMember_{counter}",
# )
#     ensemble[counter].initialize(config_file)

In [ ]:
# N = 100  # Aantal te genereren oplossingen per iteratie
# num_iterations = 10  # Hoeveel iteraties je wilt uitvoeren

# # Stap 2: Iteratief proces
# # current_population = np.random.uniform(p_min, p_max, (num_samples, len(p_min)))
# parameters = np.random.uniform(p_min_initial[:, np.newaxis], p_max_initial[:, np.newaxis], (len(p_min_initial), N))

# for iteration in range(num_iterations):
#     objectives = []

# #loop over all ensemble members
#     for ensembleMember in ensemble:
#         Q_m = []
#         time = []
#         while ensembleMember.time < ensembleMember.end_time:
#             ensembleMember.update()
#             discharge_this_timestep = ensembleMember.get_value("Q")
#             Q_m.append(discharge_this_timestep[0])
#             time.append(pd.Timestamp(ensembleMember.time_as_datetime.date()))

#     #calculate the objective function 
#     discharge_dataframe = pd.DataFrame({'model output': Q_m}, index=pd.to_datetime(time))
#     objective_this_model = calibrationObjective(discharge_dataframe,Q_pandas["Q"],calibration_start_time,calibration_end_time)
#     objectives.append(objective_this_model)

#     #it is good practice to remove any variable you don't need anymore to save memory.
#     del Q_m, time, discharge_dataframe, objective_this_model
    
#     # Stap 3: Bereken een fictieve 'objective score' (kan je vervangen met jouw functie)
#     # objectives = np.sum(parameters, axis=1)  # Voorbeeld: som van de elementen

#     # Stap 4: Selecteer de beste 10%
#     n = int(len(objectives) * 0.1)
#     best_indices = np.argsort(objectives)[:n]  # Indexen van laagste 10%
#     print((best_indices))
#     best_solutions = objectives[best_indices]  # Selecteer bijbehorende arrays
#     print(best_solutions)

#     # Stap 5: Bereken min en max per kolom uit de beste oplossingen
#     new_p_min = np.min(best_solutions, axis=0)
#     new_p_max = np.max(best_solutions, axis=0)

#     # Stap 6: Genereer nieuwe populatie tussen nieuwe min-max
#     parameters = np.random.uniform(new_p_min, new_p_max, (num_samples, len(p_min_initial)))

#     print(f"Iteratie {iteration + 1}: Nieuwe min-max ranges berekend")
#     print("Nieuwe min waarden:", new_p_min)
#     print("Nieuwe max waarden:", new_p_max)
#     print("-" * 50)



In [ ]:
#finaly, just like before, we remove the models themselves to save up space and memory.
for ensembleMember in ensemble:
    ensembleMember.finalize()

## Analyse results
We now have objective function results for all ensemble members! Let's make some plots!

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
xFigNr = 2
yFigNr = 4

fig, axs = plt.subplots(xFigNr, yFigNr,figsize = (15,15))

for xFig in range(xFigNr):
    for yFig in range(yFigNr):
        paramCounter = xFig*yFigNr + yFig
        axs[xFig,yFig].plot(parameters[paramCounter,:],objectives,'.')
        axs[xFig,yFig].set_title(p_names[paramCounter])


In [ ]:
#let's also show the minimal values:
parameters_minimum_index = np.argmin(np.array(objectives))

objective_minimum = np.min(objectives)

parameters_minimum = parameters[:,parameters_minimum_index]

print("Best parameter index:", parameters_minimum_index)
print("Best parameter set:", parameters_minimum)
print("Objective: ", objective_minimum)


In [ ]:
p_min_initial= np.array([0,   0.2,  40,    0.5,   0.001,   1,     0.01,  0.0001,  0.01])
p_max_initial = np.array([10,    3,  1000,   6,    .6,     15,    .3,   .03,  0.8])